<a href="https://colab.research.google.com/github/JonasCGN/project_facial_recognition/blob/main/Reconhecimento_Facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visão Computacional: O Guia Completo - Reconhecimento Facial

In [1]:
!pip install opencv-contrib-python==4.5.5.64

##Carregamento da base de dados

In [2]:
from PIL import Image
import cv2
print(cv2.__version__)
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

4.5.5
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
path = '/content/drive/MyDrive/Visao_Computacional_Guia_Completo/Datasets/yalefaces.zip'
zip_objetc = zipfile.ZipFile(file=path, mode='r')
zip_objetc.extractall('./')
zip_objetc.close()

##Pre-processamento das imagens

In [5]:
import os
print(os.listdir('/content/yalefaces/train'))

['subject06.normal.gif', 'subject07.surprised.gif', 'subject02.surprised.gif', 'subject09.noglasses.gif', 'subject07.sad.gif', 'subject08.happy.gif', 'subject06.wink.gif', 'subject14.centerlight.gif', 'subject04.happy.gif', 'subject15.wink.gif', 'subject06.noglasses.gif', 'subject13.rightlight.gif', 'subject14.happy.gif', 'subject05.noglasses.gif', 'subject01.glasses.gif', 'subject02.glasses.gif', 'subject15.happy.gif', 'subject05.normal.gif', 'subject03.rightlight.gif', 'subject12.wink.gif', 'subject06.rightlight.gif', 'subject15.centerlight.gif', 'subject12.centerlight.gif', 'subject07.centerlight.gif', 'subject04.glasses.gif', 'subject15.noglasses.gif', 'subject11.leftlight.gif', 'subject09.wink.gif', 'subject12.surprised.gif', 'subject09.happy.gif', 'subject10.normal.gif', 'subject03.wink.gif', 'subject07.rightlight.gif', 'subject08.wink.gif', 'subject02.noglasses.gif', 'subject13.happy.gif', 'subject11.centerlight.gif', 'subject11.sleepy.gif', 'subject04.centerlight.gif', 'subject

In [6]:
def get_image_data():
  paths = [os.path.join('/content/yalefaces/train', f) for f in os.listdir('/content/yalefaces/train')]
  #print(paths)
  faces = []
  ids = []
  for path in paths:
    imagem = Image.open(path).convert('L')
    imagem_np = np.array(imagem,'uint8')
    id = int(os.path.split(path)[1].split('.')[0].replace('subject',''))
    ids.append(id)
    faces.append(imagem_np)

  return np.array(ids), faces


In [7]:
ids, faces = get_image_data()

In [9]:
 lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
 lbph_classifier.train(faces,ids)
 lbph_classifier.write('lbph_classifier.yml')

In [ ]:
 lbph_classifier = cv2.face.LBPHFaceRecognizer_create()


##Treinamento do classificador LBPH

##Reconhecimento de faces